In [95]:
autumn['DATE'] = pd.to_datetime(autumn['DATE'])

In [2]:
autumn = pd.read_csv('autumn2.csv')

In [4]:
autumn['RANGE'] = autumn['TMAX'] - autumn['TMIN']

In [5]:
#maybe this is the database?
conn = sqlite3.connect('weather_data.db')
autumn.to_sql('lou_weather',conn)

/Users/ryanpierson/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2127: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


ValueError: Table 'lou_weather' already exists.

In [8]:
autumn.loc[autumn['RANGE'].idxmax()]
autumn.loc[autumn['RANGE'].idxmin()]

DATE     2010-10-09
TMAX             87
TMIN             47
RANGE            40
YEAR           2010
MONTH            10
WEEK             40
DAY               9
Name: 38, dtype: object

In [160]:
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import pandas as pd
import sqlite3

output_notebook()
#Gets the distinct years from the database
def main():
    conn = sqlite3.connect('weather.db')
    c = conn.cursor()
    c.execute('''CREATE TEMPORARY TABLE maxminall AS Select MONTH || DAY concat, MONTH, DAY,MAX(TMAX) over_max, MIN(TMIN) over_min 
                from weather GROUP BY 1, 2, 3 ORDER BY 2, 3''')
    #Retrieves the range for the plots
    rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
    mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])
    
    display_all_plots(sql_dict())
    

# # Query to combine the overall metrics with the individual table
# def quer(y):
#     return'''
#     select a.DATE, 
#     (AVG(a.TMAX)+ AVG(a.TMIN))/2 avg_temp,  
#     a.YEAR, 
#     a.MONTH, 
#     a.DAY, 
#     (b.over_max + b.over_min)/2 comp  
#     from weather a join maxminall b
#     on a.MONTH || a.DAY = b.concat WHERE a.MONTH > 8 AND a.YEAR = ''' + str(y) + " GROUP BY 1, 3, 4, 5, 6"


# Query to combine the overall metrics with the individual table
def quer(y):
    return'''
    select a.DATE, 
    AVG(a.TMAX) TMAX,
    AVG(a.TMIN) TMIN,  
    a.YEAR, 
    a.MONTH, 
    a.DAY, 
    b.over_max overmax, 
    b.over_min overmin  
    from weather a join maxminall b
    on a.MONTH || a.DAY = b.concat WHERE a.MONTH > 8 AND a.YEAR = ''' + str(y) + " GROUP BY 1, 4, 5, 6, 7, 8"





#Creates the max/min/all temp table


# New Version
def sql_dict():
    #query = "Select Distinct YEAR from weather WHERE YEAR > (strftime('%Y','now') - 10) ORDER BY YEAR"
    query = "Select Distinct YEAR from weather WHERE YEAR > 2010 ORDER BY YEAR"
    yr = pd.read_sql(query, conn)
    print("part two:", yr)
    sql_results = {}
    for i in range(len(yr) - 1): 
        y = yr.loc[i,'YEAR']
        #print(y)
        sql_results[str(y)] = {}
        sql_results[str(y)]['DATA'] = pd.read_sql(quer(y), conn)
        sql_results[str(y)]['DATA']['DATE'] = pd.to_datetime(sql_results[str(y)]['DATA']['DATE'])
    #print(sql_results)
    return sql_results

#makes the plot
#works
def plot_maker(data, div):
    rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
    mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])
    p = figure(x_axis_type='datetime',y_range=(mn,mx) ,width=500, plot_height=250, title=div)
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_color='red'
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = "Temperature"
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].TMAX, line_width=2, color='red')
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].TMIN, line_width=2, color='blue')
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].overmax, line_width=2, color='green', alpha=0.5)
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].overmin, line_width=2, color='purple', alpha=0.5)
    return p

# Grouping 
def data_grid(lst, col=2):
    output = []
    for i in range(0, len(lst), col):
        if (i + (col - 1)) >= len(lst):
            output.append(lst[i:])
        else:
            output.append(lst[i:(i+(col))])
    return output


def display_all_plots(data):
    viz = []
    for i in data.keys():
        print(i)
        viz.append(plot_maker(data, i))
    show(gridplot(data_grid(viz)))
    #show(data_grid(viz))

Loading BokehJS ...

In [161]:
main()

part two:    YEAR
0  2011
1  2012
2  2013
3  2014
4  2015
5  2016
6  2017
7  2018
2011
2012
2013
2014
2015
2016
2017


In [200]:
# # plot maker works -- displays the graph
# show(plot_maker(data=sql_results, div='2010'))

### -----
from bokeh.layouts import gridplot

viz = []
for i in sql_results.keys():
    viz.append(plot_maker(sql_results, i))
show(gridplot(data_grid(viz)))

In [68]:
df = sql_dict()

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
{'2002': {'DATA':           DATE       TMAX       TMIN  YEAR  MONTH  DAY  over_max  over_min
0   2002-09-01  89.666667  69.000000  2002      9    1     101.0      53.0
1   2002-09-02  91.333333  68.666667  2002      9    2     101.0      54.0
2   2002-09-03  95.333333  72.000000  2002      9    3     102.0      53.0
3   2002-09-04  91.333333  70.666667  2002      9    4      98.0      54.0
4   2002-09-05  91.666667  60.333333  2002      9    5      99.0      49.0
5   2002-09-06  94.333333  60.333333  2002      9    6      95.0      49.0
6   2002-09-07  94.000000  68.333333  2002      9    7      98.0      48.0
7   2002-09-08  96.000000  67.666667  2002      9    8      97.0      50.0
8   2002-09-09  97.000000  68.333333  2002      9    9      98.0      52.0
9   2002-09-10  96.666667  68.000000  2002      9   10      98.0      52.0
10  2002-09-11  82.333333  65.666667  2002      9   11      93.0      52.0
11

In [20]:
df

{'2010': {'DATA':      index       DATE  TMAX  TMIN  RANGE  YEAR  MONTH  WEEK  DAY
  0        0 2010-09-01    93    68     25  2010      9    35    1
  1        1 2010-09-02    95    67     28  2010      9    35    2
  2        2 2010-09-03    82    64     18  2010      9    35    3
  3        3 2010-09-04    76    54     22  2010      9    35    4
  4        4 2010-09-05    80    49     31  2010      9    35    5
  5        5 2010-09-06    91    56     35  2010      9    36    6
  6        6 2010-09-07    95    65     30  2010      9    36    7
  7        7 2010-09-08    84    63     21  2010      9    36    8
  8        8 2010-09-09    80    56     24  2010      9    36    9
  9        9 2010-09-10    72    61     11  2010      9    36   10
  10      10 2010-09-11    81    64     17  2010      9    36   11
  11      11 2010-09-12    83    58     25  2010      9    36   12
  12      12 2010-09-13    89    56     33  2010      9    37   13
  13      13 2010-09-14    90    62     28  20

In [50]:
import sqlite3

conn = sqlite3.connect('weather.db')

In [51]:
pd.read_sql("select * from weather limit 5", conn)

,index,STATION,DATE,TMAX,TMIN
0,15119,USW00093821,2002-08-01,97.0,73.0
1,15120,USW00093821,2002-08-02,96.0,76.0
2,15121,USW00093821,2002-08-03,100.0,75.0
3,15122,USW00093821,2002-08-04,101.0,74.0
4,15123,USW00093821,2002-08-05,100.0,74.0


In [76]:
def query(sql):
    return pd.read_sql(sql, conn, index_col='index')

In [236]:
# Query to combine the overall metrics with the individual table
def quer(y):
    return'''select a.DATE, 
    a.TMAX, 
    a.TMIN,  
    a.YEAR, 
    a.MONTH, 
    a.DAY, 
    b.over_max, 
    b.over_min 
    from autumn a join maxminall b
    on a.MONTH || a.DAY = b.concat WHERE YEAR = ''' + str(y)

#pd.read_sql(quer, conn)

In [59]:
conn = sqlite3.connect('weather.db')
c = conn.cursor()
c.execute('''CREATE TEMPORARY TABLE maxminall AS Select MONTH || DAY concat, MONTH, DAY,MAX(TMAX) over_max, MIN(TMIN) over_min 
            from weather GROUP BY 1, 2, 3 ORDER BY 2, 3''')
#Retrieves the range for the plots
rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])